## 新华社抄袭判断KNN

In [2]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
import jieba

In [3]:
data=pd.read_csv('/Users/yangsong/Documents/dataset/sqlResult_1558435.csv',encoding='gb18030')

In [4]:
data=data.dropna(subset=['source','content'])

In [5]:
data.loc[data.source !='新华社', 'source'] = 0
data.loc[data.source =='新华社', 'source'] = 1

In [6]:
data=shuffle(data)
data=data[:5000]

In [7]:
def cut(string):
    string=string.replace("新华社",'').replace('照片','')
    return ' '.join(jieba.cut(string))

In [8]:
content= data['content'].apply(cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/cc/hjgk_hm53ms4j5rjmy7d41gm0000gn/T/jieba.cache
Loading model cost 0.781 seconds.
Prefix dict has been built succesfully.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)
Y = data['source']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=45)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
def get_performance(clf, X_, y_):
    y_hat = clf.predict(X_.toarray())
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

In [ ]:
#使用word2vec讲句子中每个单词求词向量后求相加求平均表示句子向量
from gensim.models import Word2Vec
import re
from sklearn.neighbors import KNeighborsClassifier
model_path='/Users/yangsong/Documents/jupyter/model/news.model'
model = Word2Vec.load(model_path) #加载一个词向量模型

In [ ]:
def word_to_sentence_embedding(sententce):
    #句子切分成单词组
    words_list=list(jieba.cut(content=re.sub('[+——() ? 【】“”！，：。？、~@#￥%……&*（）《 》]+', '', sententce)))
    sententce_embedding=sum([model[w] w  for w in words_list]) #求矩阵和
    return sententce_embedding/len(words_list) #求平均

In [ ]:
X=data['content'].apply(word_to_sentence_embedding)
Y = data['source']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=45)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)
neigh.fit(X, y) 

In [ ]:
get_performance(neigh,X_train,y_train)

In [ ]:
get_performance(neigh,X_test,y_test)